In [8]:
import numpy as np 
import pandas as pd 
import nltk 
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [9]:
df=pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [10]:
pd.set_option('display.max_colwidth', 100)
df.review

0        One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked....
1        A wonderful little production. <br /><br />The filming technique is very unassuming- very old-ti...
2        I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air...
3        Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his...
4        Petter Mattei's "Love in the Time of Money" is a visually stunning film to watch. Mr. Mattei off...
                                                        ...                                                 
49995    I thought this movie did a down right good job. It wasn't as creative or original as the first, ...
49996    Bad plot, bad dialogue, bad acting, idiotic directing, the annoying porn groove soundtrack that ...
49997    I am a Catholic taught in parochial elementary schools by nuns, taught by Jesuit priests in high...
49998    I'm going 

In [11]:
df

,review,sentiment
0,One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked....,positive
1,A wonderful little production. <br /><br />The filming technique is very unassuming- very old-ti...,positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air...",positive
3,Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei off...",positive
...,...,...
49995,"I thought this movie did a down right good job. It wasn't as creative or original as the first, ...",positive
49996,"Bad plot, bad dialogue, bad acting, idiotic directing, the annoying porn groove soundtrack that ...",negative
49997,"I am a Catholic taught in parochial elementary schools by nuns, taught by Jesuit priests in high...",negative
49998,I'm going to have to disagree with the previous comment and side with Maltin on this one. This i...,negative


In [12]:
#Summary of the dataset
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not solely cooking (which would have been great too)....,positive
freq,5,25000


In [13]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [14]:
df['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [15]:
#split the dataset  
#train dataset
train_reviews=df.review[:40000]
train_sentiments=df.sentiment[:40000]
#test dataset
test_reviews=df.review[40000:]
test_sentiments=df.sentiment[40000:]
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)

(40000,) (40000,)
(10000,) (10000,)


In [16]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

In [17]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply thisfunction on review column
df['review']=df['review'].apply(remove_stopwords)

{"shouldn't", 'i', 'aren', 'on', 'any', 'above', 'hasn', 'll', 'its', 'it', "you'd", "hadn't", 'few', 'of', "it's", 'for', "needn't", 'is', 'once', 've', 'with', 'because', 'yourselves', 'then', 'but', 'has', 'theirs', 'these', 'whom', 'so', 'our', 'me', 'some', 'yourself', 'should', "mightn't", 'won', "that'll", 'we', 'in', 'needn', 'a', 'which', 'below', "couldn't", 'further', 'itself', 'mightn', 'ain', 'does', 'shouldn', 'are', 'can', 'were', 'or', 'how', 'was', 'had', "isn't", 'ourselves', 's', 'd', 'myself', 'she', 'themselves', 'about', 'this', 'mustn', 'himself', "she's", 'that', 'ours', 'do', 'at', 'their', 'no', "haven't", "won't", 'own', 'doesn', 'into', 'herself', 'after', 'here', 'when', 'to', 'just', "don't", 'his', 'couldn', 'haven', 'her', 'over', 'again', 'will', "you'll", 'very', 'o', 'your', 'they', 'why', 'than', 'didn', 'same', "you've", 'who', 'nor', 'y', 'ma', 'from', 'all', "should've", 'through', 'more', 'now', 'there', 'weren', 'm', 'between', 't', 'where', 'am

In [18]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import string
from nltk.stem import WordNetLemmatizer
word_lemmatizer = WordNetLemmatizer()

In [19]:
def preprocess_data(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\[.*?\]', '', text)
    text = text.lower()
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('\n', '', text)
    text = ' '.join(word_lemmatizer.lemmatize(i).lower() for i in text.split()) #lemmatize words
  
    
    return text

In [20]:
df['review']=df['review'].apply(preprocess_data)

In [21]:
df

,review,sentiment
0,one reviewer mentioned watching oz episode hooked right exactly happened mebr br the first thing...,positive
1,wonderful little production br br the filming technique unassuming oldtimebbc fashion give comfo...,positive
2,thought wonderful way spend time hot summer weekend sitting air conditioned theater watching lig...,positive
3,basically family little boy jake think zombie closet amp parent fighting timebr br this movie sl...,negative
4,petter mattei love time money visually stunning film watch mr mattei offer u vivid portrait huma...,positive
...,...,...
49995,thought movie right good job creative original first expecting be whole lotta fun think like com...,positive
49996,bad plot bad dialogue bad acting idiotic directing annoying porn groove soundtrack ran continual...,negative
49997,catholic taught parochial elementary school nun taught jesuit priest high school amp college sti...,negative
49998,going disagree previous comment side maltin one second rate excessively vicious western creak gr...,negative


In [22]:
#normalized train reviews
norm_train_reviews=df.review[:40000]
norm_train_reviews[0]

'one reviewer mentioned watching oz episode hooked right exactly happened mebr br the first thing struck oz brutality unflinching scene violence set right word go trust show faint hearted timid show pull punch regard drug sex violence hardcore classic use wordbr br it called oz nickname given oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inwards privacy high agenda em city home many aryan muslim gangsta latino christian italian irish scuffle death stare dodgy dealing shady agreement never far awaybr br i would say main appeal show due fact go show dare forget pretty picture painted mainstream audience forget charm forget romance oz mess around first episode ever saw struck nasty surreal say ready watched developed taste oz got accustomed high level graphic violence violence injustice crooked guard sold nickel inmate kill order get away well mannered middle class inmate turned prison bitch due lack street skill pris

In [23]:
#Normalized test reviews
norm_test_reviews=df.review[40000:]
norm_test_reviews[45005]

'read review watching piece cinematic garbage took least page find somebody else think appallingly unfunny montage acme humour indeed era least funny set sketch comedy ever seen till come along half skit already done infinitely better act monty python woody allen say nice piece animation last second highlight film would still get close summing mindless drivelridden waste minute is seminal comedy world seminal really mean semen scatological humour world scat actually feces precursor joke mean handbook comedy tit bum odd beaver nice pubescent boy least one hand free found playboy exists give break early way sketch comedy going back least ten year prior way could even forgive film even made gunpoint retro hardly sketch clown subtly perverting child may cutting edge circle could actually funny come really quite sad kept going throughout entire minute sheer belief may saved genuinely funny skit end gave film lower score recommend insomniac coma patient perhaps people suffering lockjaw jaw w

In [24]:
#Tfidf vectorizer
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(norm_train_reviews)
#transformed test reviews
tv_test_reviews=tv.transform(norm_test_reviews)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

Tfidf_train: (40000, 6270490)
Tfidf_test: (10000, 6270490)


In [25]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [26]:
df['sentiment']=le.fit_transform(df['sentiment'])

In [27]:
df

,review,sentiment
0,one reviewer mentioned watching oz episode hooked right exactly happened mebr br the first thing...,1
1,wonderful little production br br the filming technique unassuming oldtimebbc fashion give comfo...,1
2,thought wonderful way spend time hot summer weekend sitting air conditioned theater watching lig...,1
3,basically family little boy jake think zombie closet amp parent fighting timebr br this movie sl...,0
4,petter mattei love time money visually stunning film watch mr mattei offer u vivid portrait huma...,1
...,...,...
49995,thought movie right good job creative original first expecting be whole lotta fun think like com...,1
49996,bad plot bad dialogue bad acting idiotic directing annoying porn groove soundtrack ran continual...,0
49997,catholic taught parochial elementary school nun taught jesuit priest high school amp college sti...,0
49998,going disagree previous comment side maltin one second rate excessively vicious western creak gr...,0


In [28]:
firstcol=df['review']
secondcol=df['sentiment']

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
firstcol_train,firstcol_test,secondcol_train,secondcol_test=train_test_split(firstcol,secondcol,random_state=0,test_size=0.2)

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
max_features=5000
vectorizer = TfidfVectorizer(max_features=5000,sublinear_tf=True,ngram_range=(1, 3),
                             encoding='utf-8',decode_error='ignore')


vectorizer.fit(firstcol_train)
firstcol_train_trans=vectorizer.transform(firstcol_train)
firstcol_test_trans=vectorizer.transform(firstcol_test)

In [32]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()

In [33]:
lr.fit(firstcol_train_trans,secondcol_train)
pred_secondcol=lr.predict(vectorizer.transform(firstcol_test))

In [34]:
pred_secondcol

array([1, 0, 1, ..., 1, 0, 0])

In [35]:
firstcol_test

11841    john cassavetes run law bottom heap see negro sidney poitier equal quickly become friend forming...
19602    movie lame that movie unnecessary need another western western afroamericans title role sound st...
45519    well ethel water sammy davis jr billed without jr rufus jones president would one worst represen...
25747    find alan jacob review accurate concerning movie however opportunity rent dvd blockbuster commen...
42642    movie simply awesome hilarious although skating montage played comedy awesome raab brandon dicam...
                                                        ...                                                 
25091    terrible script manage attract cast ridiculous predictable thoroughly unbelievable wellacted sli...
27853    invited view film small art museum screening clue expect initially optimistic opening credit see...
47278    first detective story légitime défense belgian stanislas andré steeman whose l assassin habite a...
37020    movie grab

In [36]:
from sklearn.metrics import accuracy_score
score_1=accuracy_score(secondcol_test,pred_secondcol)

In [37]:
score_1


0.8878

In [40]:
text =["easy to give high rating"]
p=lr.predict(vectorizer.transform(text))

In [41]:
p

array([1])

In [43]:
text =["bad rating"]
p=lr.predict(vectorizer.transform(text))

In [44]:
p

array([1])